<a href="https://colab.research.google.com/github/souhirbenamor/EPF/blob/main/load_forecast_data_preprocessing_bridging_paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from datetime import datetime, timedelta

# Load Data
path = ""
name_load = "/content/Load_DayAheadForecastAndActual_DL-LU_2016to2019.xlsx"
sheet_load = "Sheet1"
name_holiday = "/content/holiday_germany_2016to2019.xlsx"
sheet_holiday = "holiday_nation"

# Read holiday data
holiday_df = pd.read_excel(path + name_holiday, sheet_name=sheet_holiday, parse_dates=["Time"])
holidays = set(holiday_df["Time"].dt.date)

# Read load data
load_df = pd.read_excel(path + name_load, sheet_name=sheet_load, parse_dates=["Time"])
load_df.columns = ["Time", "Loadforecast", "Load"]
load_df["Error"] = load_df["Load"] - load_df["Loadforecast"]
load_df.set_index("Time", inplace=True)

# Handle missing values
load_df["Loadforecast"] = load_df["Loadforecast"].ffill()
load_df["Load"] = load_df["Load"].ffill()
load_df["Error"] = load_df["Load"] - load_df["Loadforecast"]

# Define rolling window (1 year)
l_rw = timedelta(days=365)

# Forecasting loop
forecast_results = []
start_time = datetime(2016, 1, 1)
end_time = load_df.index[-1]
current_time = start_time + l_rw

while current_time <= end_time:
    # Define training data (1-year rolling window)
    train_start = current_time - l_rw
    train_end = current_time - timedelta(hours=1)
    train_data = load_df.loc[train_start:train_end]

    # Deseasonalization (weekdays, holidays, hours)
    train_data = train_data.copy()
    train_data = train_data.asfreq('h')
    train_data["Weekday"] = train_data.index.weekday
    train_data["Hour"] = train_data.index.hour
    train_data["Holiday"] = train_data.index.to_series().apply(lambda d: int(d.date() in holidays))

    # Train SARIMA model
    model = SARIMAX(train_data["Error"], order=(1, 0, 1), seasonal_order=(1, 0, 1, 24))
    model_fit = model.fit(disp=False)

    # Forecast next 24 hours
    forecast_start = current_time
    forecast_end = current_time + timedelta(hours=23)
    forecast_index = pd.date_range(start=forecast_start, end=forecast_end, freq='h')
    forecast_values = model_fit.forecast(steps=24)

    # Store results
    forecast_df = pd.DataFrame({
        "Time": forecast_index,
        "Forecast_Deseasonal": forecast_values.values
    })
    forecast_results.append(forecast_df)

    # Move to next day
    current_time += timedelta(days=1)

# Combine forecasts
forecast_df = pd.concat(forecast_results, ignore_index=True)
forecast_df.set_index("Time", inplace=True)

# Calculate performance metrics
mse_old = np.mean(load_df["Error"]**2)
mse_new = np.mean((load_df.loc[forecast_df.index, "Error"] - forecast_df["Forecast_Deseasonal"])**2)
mae_old = np.mean(np.abs(load_df["Error"]))
mae_new = np.mean(np.abs(load_df.loc[forecast_df.index, "Error"] - forecast_df["Forecast_Deseasonal"]))

# Display improvement
print("MSE Old:", mse_old, "MSE New:", mse_new)
print("MAE Old:", mae_old, "MAE New:", mae_new)

# Plot actual vs forecast
plt.figure(figsize=(12,6))
plt.plot(load_df.index, load_df["Load"], label="Actual Load", alpha=0.6)
plt.plot(forecast_df.index, forecast_df["Forecast_Deseasonal"], label="Forecast", linestyle="dashed")
plt.legend()
plt.title("Load Forecasting using SARIMA")
plt.show()
